<a href="https://colab.research.google.com/github/OlegPozovnoy/MailRuMLHomeworks/blob/master/HW3/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/datasets/')

Drive already mounted at /datasets/; to attempt to forcibly remount, call drive.mount("/datasets/", force_remount=True).


In [2]:
!ls "/datasets/My Drive/datasets/"

players.pkl  results.pkl  tournaments.pkl


In [0]:
import pickle
import pandas as pd

In [0]:
tournaments = pd.read_pickle("/datasets/My Drive/datasets/tournaments.pkl")
#tournaments = pd.read_pickle("./tournaments.pkl")

tournaments_cleaned = [(v['id'], v['name'], v['dateStart'][0:4], v['type']['name'], v['questionQty']) for k,v in tournaments.items() if v['questionQty'] != None]
#del tournaments

Создадим трейн и тест набор турниров.

In [5]:
train_id = [ v[0] for v in tournaments_cleaned if int(v[2])== 2019]
test_id = [v[0] for v in tournaments_cleaned if int(v[2])== 2020]

print(len(train_id))
print(len(test_id))

687
418


In [0]:
results = pd.read_pickle("/datasets/My Drive/datasets/results.pkl")
#results = pd.read_pickle("./results.pkl")
results_cleaned = {k: v for k,v in results.items() if k in train_id or k in test_id}

results_cleaned2 = {}
for k,v in results_cleaned.items():
  results_cleaned2[k] = []
  for s in v:
    if s.get('mask') != None:
      results_cleaned2[k].append((s.get('mask'),s.get('questionsTotal'), [k.get('player').get('id') for k in s.get('teamMembers')]))

del results
del results_cleaned

In [7]:
train_id = [id for id in train_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
#test_id = [id for id in test_id if id in results_cleaned2.keys() and len(results_cleaned2[id]) > 0]
#в test Id нас интересуют только позиции, так что турниры у которых нет масок можно не фильтровать

print(len(train_id))
print(len(test_id))

675
418


In [11]:
import numpy as np
from sklearn.linear_model import LogisticRegression

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


logistic = LogisticRegression(fit_intercept = False) #penalty = 'none', max_iter = 200

tournaments_data = {}
# запишем сюда все регрессии т.к. будем вычисления переиспользовать
def process_tournament(id):
  global tournaments_data
  task3 = []
  words = []
   
  for n in range(len(results_cleaned2[id])): # command id
    for p in range(len(results_cleaned2[id][n][0])): #answers
        quest = str(id)+'_'+str(p)
        team = n
        task3.append((quest, team,results_cleaned2[id][n][0][p]) )
        words.append(quest)
        words.append(team)

  if len(words)==0:
    return 
  
  words = set(words) # encoding questions and teams no

  idx_to_word = dict(enumerate(words))
  word_to_idx = dict([(y,x) for x,y in idx_to_word.items()])
  #preparing matrix
  X = np.zeros((len(task3),len(words)))
  Y = np.zeros(len(task3))
  for i in range(len(task3)):
    X[i,word_to_idx[task3[i][0]]] = 1
    X[i,word_to_idx[task3[i][1]]] = 1
    Y[i] = 1 if str(task3[i][2]) == '1' else 0

  if len(set(Y)) <= 1:
    print ('<2 categories , skipping')
    return

  #print(X.shape)
  logistic.fit(X, Y)
  # запишем вычисленные значения
  tournaments_data[id] = {}
  tournaments_data[id]['coef'] = logistic.coef_[0].copy()
  tournaments_data[id]['idx_to_word'] = idx_to_word.copy()
  tournaments_data[id]['word_to_idx'] = word_to_idx.copy()  
  tournaments_data[id]['task3'] = task3.copy()  

  for s in range(len(logistic.coef_[0])):
    if isinstance(idx_to_word[s], int): ## teanm no
        #print('team '+ str(idx_to_word[s]))
        for player in results_cleaned2[id][idx_to_word[s]][2]:
            #print(player)
            all_results[player][1] +=1
            all_results[player][0] +=logistic.coef_[0][s]
    else: #question no
        #print('question '+ str(idx_to_word[s]))
        all_results[idx_to_word[s]][1] +=1
        all_results[idx_to_word[s]][0] +=logistic.coef_[0][s]


104707


Пройдемся по всем турнирам и посчитаем рейтинг всех игроков и вопросов в заданном турнире, за неимением лучшего, усредним по каждому игроку рейтинг в турнире. Взвешивать на количество вопросов наверное смысла нет, т.к. на одном турнире человек находится примерно в одной и той же форме.

In [12]:
import tqdm

for tourn_id in tqdm.tqdm(train_id):
  process_tournament(tourn_id)

 99%|█████████▉| 671/675 [01:59<00:00, 27.96it/s]

<2 categories , skipping


100%|██████████| 675/675 [02:50<00:00,  3.97it/s]


In [13]:
import statistics as stat

def is_player(t):
  return isinstance(t, int)

all_ranking = {}

for k,v in all_results.items():
  if v[1] != 0:
    all_ranking[k] = v[0]/v[1] 

# blank_rank - рейтинг неизвестного игрока, считаем как ср. арифмитическое всех, хотя это наверное оптимистично
blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
print(blank_rank)


-0.5379797641338597


In [0]:
# возвращаем рейтинг команды по набору игроков
# так как число игроков мы не учитывали, пока логично возворащать среднее арифметическое всех рангов
def get_team_ranking(players):
  global all_ranking
  result = []
  #print('Team:',players)
  if (len(players)==0):
    return blank_rank
  for player in players:
    if (all_results[player][1]>0): # num of competetions
      #print(player, ' rating is ', all_ranking[player])
      result.append(all_ranking[player])
    else:
      result.append(blank_rank)
  return stat.mean(result)


In [15]:
from scipy import stats

spearman = []
pearson = []

test_id = [x for x in test_id if len(results_cleaned2[x])>1] ## фильтруем турниры с 1 командой

for tourn_id in test_id:
  rating = []
  ranking = []
  i=1
  for team in results_cleaned2[tourn_id]:
    rating.append(get_team_ranking(team[2]))
    ranking.append(-i)
    i+=1
  spearman.append(stats.spearmanr(rating, ranking)[0])
  pearson.append(stats.pearsonr(rating,ranking)[0])

print(spearman)
print(pearson)
print(stat.mean(spearman))
print(stat.mean(pearson))


[0.7302837415039378, 0.7769638794729264, 0.7857266542980827, 0.7571457221280683, 0.5878069812176181, 0.7219364318453322, 0.6338391968235293, 0.7020631281960853, 0.5958556798082136, 0.4168306855170206, 0.5182010463585892, 0.8440030750950487, 0.6928426044300088, 0.6432143638468918, 0.6397192764003768, 0.5945538602214889, 0.8308823529411766, 0.7972045963338911, 0.8032883294648357, 0.7004561824729891, 0.821067714289558, 0.8381972265023113, 0.7826655282623971, 0.8164776477647764, 0.6346326739079873, 0.7690484126021944, 0.755033462365203, 0.617391304347826, 0.8361712453244572, 0.787268817204301, 0.8869021782814887, 0.8590717678970162, 0.7640920641389868, 0.7668308702791461, 0.882706766917293, 0.8022405775523899, 0.748051948051948, 0.7898194214018072, 0.7693191013030302, 0.7615320790320791, 0.812891986062718, 0.7277026368207713, 0.7622377622377624, 0.9009304871373838, 0.7033580067453409, 0.8251082251082253, 0.8128977114315507, 0.8183650615901457, 0.7867509848641924, 0.8587219541924115, 0.8181

Теперь будем работать в рамках модели, когда участники команды отвечают на вопрос независимо и если кто то ответил - команда отвечает правильно, если никто - не отвечает. Будем попеременно улучшать оценки для сложностей вопросов и индивидуальных показателей всех участников

In [0]:
from scipy.special import expit
import math

def sigm_reverse(x):
  if x<0.001:
    return -6
  elif x>0.999:
    return 6
  result = -math.log(1/x-1)
  if (result >6):
    return 6
  elif(result < -6):
    return -6
  else:
    return result


# апдейтим индивидуальные рейтинги при данном командном рейтинге и рейтинге вопроса
# при апдейте смещаем рейтинги всех участников параллельно
# на величину shift
def adjust_individual_binary(players, team_X, Y):
  global all_ranking
  #print('Adjust individual rank by team rank: ', players, team_X, Y)
  prob_real = 1-expit(team_X + Y)
  if len(players) ==0:
    return
  
  # считываем текущие рейтинги игроков
  player_ratings = []
  for player in players:
    player_rating = all_ranking[player]
    player_ratings.append(player_rating)

  step = 0
  max_shift = team_X - max(player_ratings)
  min_shift = -3

  # сначала проверим, может уже сошлись
  current_shift = 0
  prob_adjusted = 1  
  for rating in player_ratings:
      prob_adjusted *= (1 - expit(player_rating + Y))
  
  while step <20 and abs(prob_real - prob_adjusted)> 0.000001:
    current_shift = (max_shift + min_shift)/2
    prob_adjusted = 1
    
    for rating in player_ratings:
      prob_adjusted *= (1 - expit(player_rating+ current_shift + Y))

    if prob_real > prob_adjusted: # слишком сильные
      max_shift = current_shift
    else:
      min_shift = current_shift

    step += 1

  result = [x + current_shift for x in player_ratings] 
  return result


# возвращаем "рейтинг команды" по индивидуальным рейтингам участников и вопросу
def compose_team(players, Y):
  global blank_rank
  global all_ranking
  #print('Get team rating by individual: ', players, team_X)
  prob_adjusted = 1
  players_ranks = []
  for player in players:
    if player in all_ranking:
      player_rating = all_ranking[player]
    else:
      player_rating = blank_rank
    players_ranks.append(player_rating)
    prob_adjusted *= (1 - expit(player_rating + Y))
  result = sigm_reverse(1-prob_adjusted) - Y 
  return result

# возвращаем рейтинг команды при ответе на "средний" вопрос
def get_team_rank(players):
  global all_ranking
  if len(players)==0:
    return sigm_reverse(1 - (1 - expit(blank_rank + blank_question)**4))
  return compose_team(players, blank_question)

blank_question = 0
def update_blank_question():
  global blank_question
  blank_question = stat.mean ([ v for k,v in all_ranking.items() if not is_player(k)])

In [0]:
def record_item(item, value):
  global all_results_upd
  all_results_upd[item][1] += 1
  all_results_upd[item][0] += value

def upd_EM(id, update_questions):
  global all_ranking
  global tournaments_data

  if not id in tournaments_data:
    print ('<2 categories , skipping')
    return

  task3 =  tournaments_data[id]['task3']
  idx_to_word = tournaments_data[id]['idx_to_word']
  word_to_idx = tournaments_data[id]['word_to_idx']

  for i in range(len(task3)):
    question = word_to_idx[task3[i][0]]
    team = word_to_idx[task3[i][1]]
    k_quest = tournaments_data[id]['coef'][question]
    k_team = tournaments_data[id]['coef'][team]
    players_list = results_cleaned2[id][task3[i][1]][2]
    if (update_questions):
      K = compose_team(players_list, all_ranking[task3[i][0]])
      record_item(task3[i][0], k_quest + k_team - K)
    else: # team members
      ranking_split = adjust_individual_binary(players_list, k_team,all_ranking[task3[i][0]])
      for j in range(len(players_list)):
        record_item(players_list[j], ranking_split[j])

In [22]:
import numpy as np
import tqdm

# creating all questions and players set
all_words = []
for k,v in results_cleaned2.items():
  for team in v: 
    for p in range(len(team[0])):
        all_words.append(str(k)+'_'+str(p))
    for m in team[2]:
        all_words.append(m)

all_words = set(all_words)
print(len(all_words))

# creating storage for competetion results
all_results = {}
for s in all_words:
  all_results[s] = [0,0] # total rating and num of competetions


104707


In [0]:
import tqdm

all_results_upd = {}
for s in all_words:
  all_results_upd[s] = [0,0] # total rating and num of competetions

def record_new_ratings():
  global all_ranking
  global blank_rank
  global all_results_upd
  for k,v in all_results_upd.items():
    if v[1] != 0:
      all_ranking[k] = v[0]/v[1] 
  print('blank_rank before',blank_rank)
  blank_rank = stat.mean ([ v for k,v in all_ranking.items() if is_player(k)])
  print('blank_rank after',blank_rank)

  all_results_upd = {}
  for s in all_words:
    all_results_upd[s] = [0,0] # total rating and num of competetions


def calc_corellations():
  spearman = []
  pearson = []

  for tourn_id in test_id:
    rating = []
    ranking = []
    i=1
    for team in results_cleaned2[tourn_id]:
      rating.append(get_team_rank(team[2]))
      ranking.append(-i)
      i+=1
    spearman.append(stats.spearmanr(rating, ranking)[0])
    pearson.append(stats.pearsonr(rating,ranking)[0])
  
  print('Spearman: ',stat.mean(spearman))
  print('Pearson: ',stat.mean(pearson))  


def EM_cycle():  
  print('Updating individual ranking')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = False)

  record_new_ratings()
  print('Updating questions')
  for tourn_id in tqdm.tqdm(train_id):
    upd_EM(tourn_id, update_questions = True)

  update_blank_question()
  calc_corellations()

#EM_cycle()

In [35]:
# сделаем 10 em итераций
for i in range(10):
  print('итерация ',i)
  EM_cycle()



 63%|██████▎   | 423/675 [09:17<16:14,  3.87s/it]

 63%|██████▎   | 424/675 [09:19<14:16,  3.41s/it]

 63%|██████▎   | 425/675 [09:20<10:37,  2.55s/it]

 63%|██████▎   | 426/675 [09:21<09:05,  2.19s/it]

 63%|██████▎   | 427/675 [09:24<10:23,  2.51s/it]

 63%|██████▎   | 428/675 [09:26<09:05,  2.21s/it]

 64%|██████▎   | 429/675 [09:27<07:15,  1.77s/it]

 64%|██████▎   | 430/675 [09:27<05:58,  1.46s/it]

 64%|██████▍   | 431/675 [09:30<07:30,  1.85s/it]

 64%|██████▍   | 432/675 [09:32<07:46,  1.92s/it]

 64%|██████▍   | 433/675 [09:33<06:04,  1.51s/it]

 64%|██████▍   | 434/675 [09:34<05:22,  1.34s/it]

 64%|██████▍   | 435/675 [09:35<05:06,  1.28s/it]

 65%|██████▍   | 436/675 [09:36<04:44,  1.19s/it]

 65%|██████▍   | 437/675 [09:37<04:55,  1.24s/it]

 65%|██████▍   | 438/675 [09:39<05:08,  1.30s/it]

 65%|██████▌   | 439/675 [09:39<03:42,  1.06it/s]

 65%|██████▌   | 440/675 [09:39<03:09,  1.24it/s]

 65%|██████▌   | 441/675 [09:41<03:53,  1.00it/s]

 65%|██████▌   | 442/675 [09:

<2 categories , skipping




 99%|█████████▊| 665/675 [13:30<00:02,  3.65it/s]

 99%|█████████▊| 666/675 [13:30<00:02,  4.06it/s]

 99%|█████████▉| 667/675 [13:31<00:01,  4.10it/s]

 99%|█████████▉| 668/675 [13:31<00:01,  3.81it/s]

 99%|█████████▉| 669/675 [13:31<00:01,  3.99it/s]

 99%|█████████▉| 670/675 [13:31<00:01,  4.04it/s]

 99%|█████████▉| 671/675 [13:32<00:01,  2.93it/s]

100%|█████████▉| 672/675 [13:32<00:00,  3.27it/s]

100%|█████████▉| 673/675 [13:54<00:13,  6.77s/it]

100%|█████████▉| 674/675 [14:29<00:15, 15.33s/it]

100%|██████████| 675/675 [15:34<00:00,  1.38s/it]


  0%|          | 0/675 [00:00<?, ?it/s]

blank_rank before -2.5258240138828736
blank_rank after -2.59017120811673
Updating questions




  0%|          | 1/675 [00:00<01:29,  7.50it/s]

  0%|          | 2/675 [00:00<02:53,  3.87it/s]

  0%|          | 3/675 [00:01<03:56,  2.85it/s]

  1%|          | 4/675 [00:01<04:30,  2.48it/s]

  1%|          | 5/675 [00:02<04:50,  2.31it/s]

  1%|          | 7/675 [00:02<03:44,  2.97it/s]

  1%|          | 8/675 [00:02<03:05,  3.60it/s]

  1%|▏         | 9/675 [00:02<02:47,  3.97it/s]

  1%|▏         | 10/675 [00:03<02:33,  4.32it/s]

  2%|▏         | 11/675 [00:03<02:21,  4.69it/s]

  2%|▏         | 12/675 [00:03<02:03,  5.36it/s]

  2%|▏         | 13/675 [00:03<02:36,  4.22it/s]

  2%|▏         | 14/675 [00:04<04:24,  2.50it/s]

  2%|▏         | 16/675 [00:04<03:28,  3.16it/s]

  3%|▎         | 17/675 [00:04<02:56,  3.73it/s]

  3%|▎         | 18/675 [00:04<02:24,  4.54it/s]

  3%|▎         | 19/675 [00:05<02:06,  5.19it/s]

  3%|▎         | 21/675 [00:05<01:43,  6.29it/s]

  3%|▎         | 23/675 [00:05<01:44,  6.25it/s]

  4%|▎         | 24/675 [00:05<02:13,  4.89it/s]

  4%|▎

...

Updating questions




  0%|          | 2/675 [00:00<02:53,  3.87it/s]

  0%|          | 3/675 [00:01<03:56,  2.84it/s]

  1%|          | 4/675 [00:01<04:34,  2.45it/s]

  1%|          | 5/675 [00:02<04:58,  2.25it/s]

  1%|          | 7/675 [00:02<03:55,  2.84it/s]

  1%|          | 8/675 [00:02<03:12,  3.47it/s]

  1%|▏         | 9/675 [00:02<02:54,  3.83it/s]

  1%|▏         | 10/675 [00:03<02:38,  4.21it/s]

  2%|▏         | 11/675 [00:03<02:23,  4.63it/s]

  2%|▏         | 12/675 [00:03<02:03,  5.36it/s]

  2%|▏         | 13/675 [00:03<02:39,  4.14it/s]

  2%|▏         | 14/675 [00:04<04:17,  2.56it/s]

  2%|▏         | 16/675 [00:04<03:21,  3.27it/s]

  3%|▎         | 17/675 [00:04<02:45,  3.98it/s]

  3%|▎         | 19/675 [00:05<02:13,  4.91it/s]

  3%|▎         | 21/675 [00:05<01:45,  6.22it/s]

  3%|▎         | 23/675 [00:05<01:45,  6.19it/s]

  4%|▎         | 24/675 [00:05<02:11,  4.95it/s]

  4%|▎         | 25/675 [00:06<02:52,  3.76it/s]

  4%|▍         | 26/675 [00:06<02:43,  3.96it/s]

  4%|

<2 categories , skipping




100%|██████████| 675/675 [01:04<00:00, 10.41it/s]


Spearman:  0.6980745104395094
Pearson:  0.6908023700884429


In [48]:
question_ranking = [ (k, v) for k,v in all_ranking.items() if not is_player(k)]
print(sorted(question_ranking, key=lambda x: x[1]))

tourn_stat = {}

for s in question_ranking:
    id = s[0].split('_')[0]
    if not id in tourn_stat:
        tourn_stat[id] = [0,0]
    tourn_stat[id][0] += s[1]
    tourn_stat[id][1] +=1
  
print(tourn_stat)
tournament_rating = [(k,v[0]/v[1], tournaments[int(k)]['name']) for k,v in tourn_stat.items() if v[1] > 0 and int(k) in tournaments]
print(sorted(tournament_rating, key=lambda x: x[1]))

df_tourn_rating = pd.DataFrame(sorted(tournament_rating, key=lambda x: x[1]), columns =['Id', 'Rating', 'Name'])

print(df_tourn_rating)

[('5819_25', -5.617282962621336), ('5757_30', -5.601034479832594), ('5757_14', -5.601034479832594), ('5757_34', -5.601034479832594), ('4986_23', -5.4896798595977065), ('5718_18', -5.343700918497844), ('5718_5', -5.343700918497844), ('5756_33', -5.192388438332711), ('5756_11', -5.192388438332711), ('5756_18', -5.192388438332711), ('5141_34', -5.1842559757119915), ('5594_12', -5.0588027762554715), ('5594_37', -5.0588027762554715), ('5025_42', -4.967504951161456), ('5025_28', -4.869991095503511), ('5821_19', -4.841924662578859), ('5097_16', -4.75972990492226), ('5097_17', -4.75972990492226), ('5097_10', -4.75972990492226), ('5070_6', -4.731998262633488), ('6255_181', -4.721397030396317), ('5516_10', -4.702613598028029), ('5025_45', -4.699380195902268), ('5310_0', -4.668775944575767), ('5310_36', -4.668775944575767), ('5025_39', -4.622747440773968), ('5764_32', -4.602721414198986), ('5098_27', -4.588571222132737), ('5401_13', -4.559070474866848), ('5186_17', -4.496485745393864), ('5549_32'

In [0]:
player_ranking = [ (k, v) for k,v in all_ranking.items() if is_player(k)]
print(sorted(player_ranking, key=lambda x: -x[1]))

df_players = pd.DataFrame(sorted(player_ranking, key=lambda x: -x[1]), columns =['Player', 'Rating'])
print(df_players[0:20]['Player'])

players = pd.read_pickle("/datasets/My Drive/datasets/players.pkl")
print([players[i] for i in df_players[0:20]['Player'] ] )

del players